In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# Load muc123a
part123a = pd.read_csv('/content/muc123a.csv')
part123a.info()

In [ ]:
# Create household size column
part123a['hsize'] = part123a.groupby(['tinh','huyen','xa','diaban','hoso'])['matv'].transform('max')

# Filter to keep only rows where m1ac3 == 1 (household head)
part123a.drop(part123a[(part123a['m1ac3'] != 1) | (part123a['m1ac2'] != 1)].index, inplace=True)

# Keep only household heads who are male and age ≥ 25
part123a = part123a[(part123a['m1ac3'] == 1) & (part123a['m1ac2'] == 1) & (part123a['m1ac5'] >= 25)]

# We take m123a as the base to merge the labels of other files
# Only keep the variables of label and individuals age
col123a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso','matv','hsize', 'm1ac2', 'm1ac3', 'm1ac5']

# Keep only the selected columns in the dataset
part123a = part123a[col123a]

# Select the columns from muc123a to create a new dataframe
df = part123a[col123a].copy()

In [ ]:
# Import income file
part4a = pd.read_csv('/content/muc4a.csv')

# Define the required columns (household identifiers + selected variables)
# m4ac7: average days work per month, m4ac8: average hours work per day
# m4ac11: cash received from main job, m4ac12f: other salary
# m4ac21: cash received from secondary job, m4ac22f: other salary 1, m4ac25: other salary 2
# m4ac27: average hours per day for secondary job
col4a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv', 'm4ac6', 'm4ac7', 'm4ac8',
           'm4ac16', 'm4ac17', 'm4ac18', 'm4ac11', 'm4ac12f', 'm4ac21', 'm4ac22f', 'm4ac25']

# Filter the dataset to keep only the selected columns
part4a = part4a[col4a]

# Create a new column that sums the selected variables for individual's income
part4a['indi_income'] = part4a[['m4ac11', 'm4ac12f','m4ac21', 'm4ac22f', 'm4ac25']].sum(axis=1)

# Group by household ID to calculate total household income
hh_income = part4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['indi_income'].sum().reset_index()

# Rename column for clarity
hh_income.rename(columns={'indi_income': 'HH_Income'}, inplace=True)

# Merge household income back into the individual-level dataset
part4a = part4a.merge(hh_income, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Fill NaN values with zero
part4a.fillna(0, inplace=True)

# Create a new dataframe to merge all files
df = df.merge(part4a, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'], how='left')

In [ ]:
df

In [ ]:
# Load household expenditure of food and drinks during holidays (5A1)
part5a1 = pd.read_csv('/content/muc5a1.csv')

# Define the required columns (household identifiers + selected variables)
# m5a1c2b: expense bought, m5a1c3b: expense self supplied or received
col5a1 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5a1c2b', 'm5a1c3b']

# Filter the dataset to keep only the selected columns
part5a1 = part5a1[col5a1]

# Group by household ID and sum the expenses
exp1 = part5a1.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[['m5a1c2b', 'm5a1c3b']].sum().reset_index()

# Create a new column for total expense
exp1['HH_exp1'] = exp1['m5a1c2b'] + exp1['m5a1c3b']

# Merge back to the original dataset
part5a1 = part5a1.merge(exp1, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Drop duplicates so that each 'hoso' appears only once
part5a1 = part5a1.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first')

part5a1

In [ ]:
# Create a new dataframe to merge all files
df = part5a1.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
# Load household daily expenditure on food and drinks
part5a2 = pd.read_csv('/content/muc5a2.csv')

# Define the required columns (household identifiers + selected variables)
# m5a2c6: expense bought, m5a2c10: expense self supplied or received
col5a2 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5a2c6', 'm5a2c10']

# Filter the dataset to keep only the selected columns
part5a2 = part5a2[col5a2]

# Group by household ID and sum the expenses
exp2 = part5a2.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[['m5a2c6', 'm5a2c10']].sum().reset_index()

# Create a new column for total expense
exp2['HH_exp2'] = exp2['m5a2c6'] + exp2['m5a2c10']

# Merge back to the original dataset
part5a2 = part5a2.merge(exp2, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Drop duplicates so that each 'hoso' appears only once
part5a2 = part5a2.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first')

part5a2

In [ ]:
# Create a new dataframe to merge all files
df = part5a2.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
# Load household daily expenditure on nonfood and others
part5b1 = pd.read_csv('/content/muc5b1.csv')

# Define the required columns (household identifiers + selected variables)
# m5b1c4: expense recieved, m5b1c5: annual expense
col5b1 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso','m5b1c4', 'm5b1c5']

# Filter the dataset to keep only the selected columns
part5b1 = part5b1[col5b1]

# Group by household ID and sum the expenses
exp3 = part5b1.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[['m5b1c4','m5b1c5']].sum().reset_index()

# Create a new column for total expense
exp3['HH_exp3'] = exp3['m5b1c4'] + exp3['m5b1c5']

# Merge back to the original dataset
part5b1 = part5b1.merge(exp3, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Drop duplicates so that each 'hoso' appears only once
part5b1 = part5b1.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first')

part5b1

In [ ]:
# Create a new dataframe to merge all files
df = part5b1.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
# Load household annual consumption expenditure
part5b2 = pd.read_csv('/content/muc5b2.csv')

# Define the required columns (household identifiers + selected variables)
# m5b2c2: expense bought, m5b2c3: expense self supplied or received
col5b2 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5b2c2', 'm5b2c3']

# Filter the dataset to keep only the selected columns
part5b2 = part5b2[col5b2]

# Group by household ID and sum the expenses
exp4 = part5b2.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[['m5b2c2', 'm5b2c3']].sum().reset_index()
part5b2
# Create a new column for total expense
exp4['HH_exp4'] = exp4['m5b2c2'] + exp4['m5b2c3']

# Merge back to the original dataset
part5b2 = part5b2.merge(exp4, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Drop duplicates so that each 'hoso' appears only once
part5b2 = part5b2.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first')

part5b2

In [ ]:
# Create a new dataframe to merge all files
df = part5b2.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
# Load other spending that is considered as household expenditure
part5b3 = pd.read_csv('/content/muc5b3.csv')

# Define the required columns (household identifiers + selected variables)
# m5b3c2: annual expense
col5b3 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5b3c2']

# Filter the dataset to keep only the selected columns
part5b3 = part5b3[col5b3]

# Group by household ID and sum the expenses
exp5 = part5b3.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[['m5b3c2']].sum().reset_index()

# Create a new column for total expense
exp5['HH_exp5'] = exp5['m5b3c2']

# Merge back to the original dataset
part5b3 = part5b3.merge(exp5, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Drop duplicates so that each 'hoso' appears only once
part5b3 = part5b3.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first')

part5b3

In [ ]:
# Create a new dataframe to merge all files
df = part5b3.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
# Load household's accomodation expenditure
part7 = pd.read_csv('/content/muc7.csv')

# Define the required columns (household identifiers + selected variables)
# m7c15: land or house/flat leasing
# m7c32: annual water expense, m7c36: annual electricity expense , m7c39: annual garbage collection expense
col7 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm7c15', 'm7c32', 'm7c36', 'm7c39']

# Filter the dataset to keep only the selected columns
part7 = part7[col7]

# Calculate total housing expense
part7['HH_exp6'] = part7['m7c32'] + part7['m7c36'] + part7['m7c39']

part7

In [ ]:
# Create a new dataframe to merge all files
df = part7.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df.info()

In [ ]:
# Import household fixed assets (6A)
part6a = pd.read_csv('/content/muc6a.csv')

# Define the required columns (household identifiers + selected variables)
# m6ac3: quantity of the assets, m6ac6: assets' value at current price, m6ac7: percentage of ownership
part6a['m6ac7'] = part6a['m6ac7'] / 100
col6a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6ac3', 'm6ac6', 'm6ac7']

# Filter the dataset to keep only the selected columns
part6a = part6a[col6a].fillna(0)

# Calculate total household fixed assets wealth
part6a['HH_wealth1'] = part6a['m6ac3'] * part6a['m6ac6'] * part6a['m6ac7']

# Aggregate total wealth by household
hh_wealth1 = part6a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'], as_index=False)['HH_wealth1'].sum()

# Import household durable goods (6B)
part6b = pd.read_csv('/content/muc6b.csv')

# Define the required columns (household identifiers + selected variables)
# m6bc3: quantity of the durable appliance, m6bc6: durable appliance value at current price
col6b = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6bc3', 'm6bc6']

# Filter the dataset to keep only the selected columns
part6b = part6b[col6b].fillna(0)

# Calculate total household durable appliance wealth
part6b['HH_wealth2'] = part6b['m6bc3'] * part6b['m6bc6']

# Aggregate total durable appliance wealth by household
hh_wealth2 = part6b.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'], as_index=False)['HH_wealth2'].sum()

# Merge wealth data into df
df = df.merge(hh_wealth1, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')
df = df.merge(hh_wealth2, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Fill missing values with 0 (in case some households have no assets or durable goods)
df[['HH_wealth1', 'HH_wealth2']] = df[['HH_wealth1', 'HH_wealth2']].fillna(0)

# Calculate total household wealth
df['HH_Wealth'] = df['HH_wealth1'] + df['HH_wealth2']

df


In [ ]:
# Import household fixed assets (6A)
part6a = pd.read_csv('/content/muc6a.csv')

# Define the required columns (household identifiers + selected variables)
# m6ac3: quantity of the assets, m6ac6: assets' value at current price , m6ac7: percentage of ownership
part6a['m6ac7'] = part6a['m6ac7']/100
col6a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6ac3', 'm6ac6', 'm6ac7']

# Filter the dataset to keep only the selected columns
part6a = part6a[col6a]
part6a.fillna(0, inplace=True)

# Calculate total household assets
part6a['HH_wealth1'] = part6a['m6ac3'] * part6a['m6ac6'] * part6a['m6ac7']

# Group by household to get total wealth per household
hh_wealth1 = part6a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'], as_index=False)['HH_wealth1'].sum()

# Merge back to the original dataset
part6a = part6a.merge(hh_wealth1, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

part6a.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first', inplace=True)

part6a

In [ ]:
# Create a new dataframe to merge all files
df = part6a.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

NameError: name 'part6a' is not defined

In [ ]:
# Import household durable goods (6B)
part6b = pd.read_csv('/content/muc6b.csv')

# Define the required columns (household identifiers + selected variables)
# m6bc3: quantity of the durable applicance, m6bc6: durable appliance value at current price
col6b = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6bc3', 'm6bc6']

# Filter the dataset to keep only the selected columns
part6b = part6b[col6b]
part6b.fillna(0, inplace=True)

# Calculate total durable goods value of household
part6b['HH_wealth2'] = part6b['m6bc3'] * part6b['m6bc6']

# Group by household to get total wealth per household
hh_wealth2 = part6b.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'], as_index=False)['HH_wealth2'].sum()

# Merge back to the original dataset
part6b = part6b.merge(hh_wealth2, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

part6b.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first', inplace=True)

part6b

In [ ]:
# Create a new dataframe to merge all files
df = part6b.merge(df, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')
df

In [ ]:
df['HH_Wealth'] = df['HH_wealth1'] + df['HH_wealth2']
df

In [ ]:
# m4ac6: working months annually (first),  m4ac7: average days work per month (first), m4ac8: average hours work per day (first)
# m4ac16: working months annually (secondary),  m4ac17: average days work per month (secondary), m4ac18: average hours work per day (secondary)
# m4ac11: cash received from main job, m4ac12f: other salary
# m4ac21: cash received from secondary job, m4ac22f: other salary 1, m4ac25: other salary 2

In [ ]:
df.to_csv('output.csv',index=False, encoding = 'utf-8-sig')

In [ ]:
df.to_csv('output2.csv',index=False, encoding = 'utf-8-sig')

In [ ]:
df

In [ ]:
# Rename columns for clarity
df = df.rename(columns={"m1ac5": "age"})
# Compute log income
df["log_income"] = np.log(df["HH_Income"].replace(0, np.nan))

# Group by age and compute average log income
log_income_by_age = df.groupby("age")["log_income"].mean().reset_index()

# Exponentiate the averaged log income
Gt = np.exp(log_income_by_age)

process_df = pd.DataFrame({
    "age": log_income_by_age["age"],
    "Gt": Gt["log_income"]
})
process_df

In [ ]:
# Plotting (with extreme ages included)
plt.figure(figsize=(10, 6))
plt.plot(process_df["age"], process_df["Gt"], color='orange', marker='o', linestyle='-')
plt.xlabel("Age")
plt.ylabel(r"$G_t$ (exp. of avg. log income)")
plt.title("Pattern of $G_t$ Across Age")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
process_df.to_csv('data3.csv',index=False, encoding = 'utf-8-sig')

In [ ]:
df.column

NameError: name 'df' is not defined

In [ ]:
# Load muc4a
m4a = pd.read_csv(os.path.join('muc4a.csv'))

# Select necessary columns
columns4a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv',
             'm4ac6', 'm4ac7', 'm4ac8',  # primary job
             'm4ac16', 'm4ac17', 'm4ac18',  # secondary job
             'm4ac11', 'm4ac12f', 'm4ac21', 'm4ac22f', 'm4ac25']

m4a = m4a[columns4a]

# Calculate primary and secondary job working hours
m4a['primary_hours'] = m4a['m4ac6'] * m4a['m4ac7'] * m4a['m4ac8']
m4a['secondary_hours'] = m4a['m4ac16'] * m4a['m4ac17'] * m4a['m4ac18']

# Total annual working hours per individual
m4a['total_hours'] = m4a[['primary_hours', 'secondary_hours']].sum(axis=1)

# Income per individual
m4a['indi_income'] = m4a[['m4ac11', 'm4ac12f','m4ac21', 'm4ac22f', 'm4ac25']].sum(axis=1)

# Remove cases: working hours > 0 but income == 0
m4a.dropna(subset=['total_hours', 'indi_income'], inplace=True)
m4a = m4a[~((m4a['total_hours'] > 0) & (m4a['indi_income'] == 0))]
m4a['is_working'] = m4a['total_hours'] > 0

# Aggregate to household level
hh_agg = m4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso']).agg(
    total_hours=('total_hours', 'sum'),
    total_income=('indi_income', 'sum'),
    num_workers=('is_working', 'sum')
).reset_index()

# Merge with household size
hh_agg = hh_agg.merge(df[['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'hsize']].drop_duplicates(),
                      on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')
hh_info = df[['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'hsize']].drop_duplicates()
hh_info = hh_info.dropna(subset=['hsize'])  #Drop missing hsize
hh_agg = hh_agg.merge(hh_info, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='inner')

# Filter households with workers and compute avg hours per worker
hh_agg = hh_agg[hh_agg['num_workers'] > 0]
hh_agg['avg_hours_per_worker'] = hh_agg['total_hours'] / hh_agg['num_workers']

# Normalize working hours (260 days x 16 hours/day)
hh_agg['normalized_hours'] = hh_agg['avg_hours_per_worker'] / (260 * 16)

# Final filter
hh_agg = hh_agg[(hh_agg['total_income'] > 0) & (hh_agg['normalized_hours'] > 0)]
print(hh_agg)

